In [1]:
import cv2
print(cv2.__version__)


4.10.0


In [ ]:
import numpy as np
import cv2

# Load the YOLOv4 model configuration and weights
yolo_model = cv2.dnn.readNetFromDarknet(r"C:\Users\kopan\OneDrive\Desktop\AI\AI Codes\yolov4.cfg", r"C:\Users\kopan\OneDrive\Desktop\AI\AI Codes\yolov4.weights")

# Use GPU if available
yolo_model.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
yolo_model.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# Load the COCO class labels
class_labels = ["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
                "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
                "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
                "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
                "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
                "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
                "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair",
                "sofa", "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote",
                "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book",
                "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]

# Declare colors for different classes
class_colors = ["0,255,0", "0,0,255", "255,0,0", "255,255,0", "0,255,255"]
class_colors = [np.array(color.split(",")).astype("int") for color in class_colors]
class_colors = np.array(class_colors)
class_colors = np.tile(class_colors, (16, 1))

# Set up video capture
video_path = r"C:\Users\kopan\OneDrive\Desktop\AI\7.Object\Images\video (1).mp4"  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# YOLOv4 layer names
layer_names = yolo_model.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo_model.getUnconnectedOutLayers()]

# Process each frame of the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_height, frame_width = frame.shape[:2]

    # Prepare the frame as blob input for YOLO
    img_blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    yolo_model.setInput(img_blob)
    obj_detection_layers = yolo_model.forward(output_layers)

    # Lists for NMS processing
    class_ids_list = []
    boxes_list = []
    confidences_list = []

    # Process detection outputs
    for detection_layer in obj_detection_layers:
        for object_detection in detection_layer:
            scores = object_detection[5:]
            predicted_class_id = np.argmax(scores)
            confidence = scores[predicted_class_id]

            if confidence > 0.5:
                box = object_detection[0:4] * np.array([frame_width, frame_height, frame_width, frame_height])
                (center_x, center_y, width, height) = box.astype("int")

                start_x = int(center_x - (width / 2))
                start_y = int(center_y - (height / 2))

                # Add the box coordinates, class ID, and confidence for NMS
                boxes_list.append([start_x, start_y, int(width), int(height)])
                confidences_list.append(float(confidence))
                class_ids_list.append(predicted_class_id)

    # Apply Non-Maxima Suppression (NMS)
    max_value_ids = cv2.dnn.NMSBoxes(boxes_list, confidences_list, 0.5, 0.4)

    # Draw the bounding boxes and labels
    if len(max_value_ids) > 0:
        for i in max_value_ids.flatten():
            (x, y) = (boxes_list[i][0], boxes_list[i][1])
            (w, h) = (boxes_list[i][2], boxes_list[i][3])
            color = [int(c) for c in class_colors[class_ids_list[i]]]
            label = f"{class_labels[class_ids_list[i]]}: {confidences_list[i]:.2f}"

            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the output frame
    cv2.imshow('YOLOv4 Video Detection', frame)

    # Press 'q' to break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
